In [582]:
import pandas as pd  # library for data analysis
import requests  # library to handle requests
from bs4 import BeautifulSoup  # library to parse HTML documents
import numpy as np


# Importing the csv file as a dataframe
df = pd.read_csv('OECD Total tax revenue data.csv')
df = df.loc[:, ['Country', 'Year', 'Value']]

# Chosing only G7 countries plus Norway and Switzerland
df = df[df['Country'].isin(['Germany', 'United Kingdom', 'France',
                           'Canada', 'United States', 'Italy', 'Japan', 'Switzerland', 'Norway'])]

df.head(7)

,Country,Year,Value
142,Canada,1965,14.210
143,Canada,1966,16.409
144,Canada,1967,18.324
145,Canada,1968,20.745
146,Canada,1969,24.723
147,Canada,1970,27.214
148,Canada,1971,30.621


In [563]:
from functools import reduce

# Inflation adjustment
# Scraping yearly inflaiton data

Inflation = pd.read_html(
    'https://www.worlddata.info/europe/united-kingdom/inflation-rates.php')

df_Inflation = Inflation[0]


df_Inflation['United Kingdom'] = df_Inflation['United Kingdom'].str.rstrip(
    '%').astype('float')
df_Inflation['Ø EU'] = df_Inflation['Ø EU'].str.rstrip('%').astype('float')
df_Inflation['Ø USA'] = df_Inflation['Ø USA'].str.rstrip(
    '%').astype('float')
df_Inflation['Ø World'] = df_Inflation['Ø World'].str.rstrip(
    '%').astype('float')

df['Year'] = df['Year'].astype(int)


df_Inflation.loc[0, 'UK index'] = 1
df_Inflation.loc[0, 'EU index'] = 1
df_Inflation.loc[0, 'USA index'] = 1

# Creating index values from the yearly inflation data
number = 1

for _ in range(61):
    df_Inflation.loc[(number), 'UK index'] = (df_Inflation.loc[(
        number-1), 'UK index'])/(1+((df_Inflation.loc[(number-1), 'United Kingdom'])/100))
    number = number + 1


number = 1
for _ in range(61):
    df_Inflation.loc[(number), 'EU index'] = (df_Inflation.loc[(
        number-1), 'EU index'])/(1+((df_Inflation.loc[(number-1), 'Ø EU'])/100))
    number = number + 1

number = 1
for _ in range(61):
    df_Inflation.loc[(number), 'USA index'] = (df_Inflation.loc[(
        number-1), 'USA index'])/(1+((df_Inflation.loc[(number-1), 'Ø USA'])/100))
    number = number + 1



dfs = [df, df_Inflation]
merged_df = reduce(lambda left, right: pd.merge(
    left, right, on=['Year'], how='inner'), dfs)


merged_df['Year'] = merged_df['Year'].astype(int)

# Making a world average which is the average of the UK, EU and USA.
merged_df["World Average"] = merged_df.loc[:, ["UK index", "EU index", "USA index"]].mean(axis=1)

merged_df = merged_df.loc[:, ['Country', 'Year', 'Value', 'World Average']]
merged_df["Adjusted Value"] = merged_df["Value"] / merged_df["World Average"]
merged_df.columns = merged_df.columns.str.replace('Value', 'Total tax revenue')
merged_df.tail(10)


,Country,Year,Total tax revenue,World Average,Adjusted Total tax revenue
460,United Kingdom,2020,885.513,0.968554,914.262487
461,United States,2020,5380.217,0.968554,5554.893689
462,Canada,2021,661.276,1.000000,661.276000
463,France,2021,1335.286,1.000000,1335.286000
464,Germany,2021,1683.080,1.000000,1683.080000
465,Italy,2021,908.944,1.000000,908.944000
466,Norway,2021,203.640,1.000000,203.640000
467,Switzerland,2021,227.231,1.000000,227.231000
468,United Kingdom,2021,1066.957,1.000000,1066.957000
469,United States,2021,6112.256,1.000000,6112.256000


In [564]:

# Create an empty list to store the data frames
df_list = []

# Defining the base URL of the website
base_url = "https://en.wikipedia.org/wiki/"

# Define a list of subdirectories
Countries_series = ["Demographics_of_Germany", "Demography_of_the_United_Kingdom", "Demographics_of_France", "Demographics_of_Canada",
                    "Demographics_of_the_United_States", "Demographics_of_Italy","Demographics_of_Switzerland", "Demographics_of_Norway"]

# Loop through the subdirectories
for Country in Countries_series:
  # Create the full URL by combining the base URL and the subdirectory
  url = base_url + Country


  population = pd.read_html(url, match="Average population")
  population
  df = pd.DataFrame(population[0])

  # Droping the second level of the index if applicable
  if df.index.nlevels > 1:
    df = df.reset_index(level=1, drop=True)

  df = df.iloc[:, :2]

  df.columns.values[0] = 'Year'
  df.columns.values[1] = 'Population'

  df['Country'] = Country


 # Append the data frame to the list
  df_list.append(df)

# Concatenate the data frames into a single data frame
  result = pd.concat(df_list, axis=1)
  


In [565]:
result.head()

,"(Unnamed: 0_level_0, Unnamed: 0_level_1)","(Average population, Average population)","(Country, )",Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country
0,1817,25009000,Demographics_of_Germany,1900.0,41154600,Demography_of_the_United_Kingdom,1900.0,NaN,Demographics_of_France,1900.0,5500000.0,Demographics_of_Canada,1935,127250000.0,Demographics_of_the_United_States,1900.0,32377000.0,Demographics_of_Italy,1900.0,3300000.0,Demographics_of_Switzerland,1900.0,2231000.0,Demographics_of_Norway
1,1818,25369000,Demographics_of_Germany,1901.0,41538200,Demography_of_the_United_Kingdom,1901.0,40710000.0,Demographics_of_France,1901.0,5600000.0,Demographics_of_Canada,1936,128053000.0,Demographics_of_the_United_States,1901.0,32550000.0,Demographics_of_Italy,1901.0,3341000.0,Demographics_of_Switzerland,1901.0,2255000.0,Demographics_of_Norway
2,1819,25733000,Demographics_of_Germany,1902.0,41892700,Demography_of_the_United_Kingdom,1902.0,40810000.0,Demographics_of_France,1902.0,5760000.0,Demographics_of_Canada,1937,128825000.0,Demographics_of_the_United_States,1902.0,32787000.0,Demographics_of_Italy,1902.0,3384000.0,Demographics_of_Switzerland,1902.0,2276000.0,Demographics_of_Norway
3,1820,26101000,Demographics_of_Germany,1903.0,42246600,Demography_of_the_United_Kingdom,1903.0,40910000.0,Demographics_of_France,1903.0,5930000.0,Demographics_of_Canada,1938,129825000.0,Demographics_of_the_United_States,1903.0,33004000.0,Demographics_of_Italy,1903.0,3428000.0,Demographics_of_Switzerland,1903.0,2288000.0,Demographics_of_Norway
4,1821,26473000,Demographics_of_Germany,1904.0,42611400,Demography_of_the_United_Kingdom,1904.0,41000000.0,Demographics_of_France,1904.0,6100000.0,Demographics_of_Canada,1939,130880000.0,Demographics_of_the_United_States,1904.0,33237000.0,Demographics_of_Italy,1904.0,3472000.0,Demographics_of_Switzerland,1904.0,2298000.0,Demographics_of_Norway


In [566]:
# The table for Japan was stored in a slightly different way, therefore broke the loop. Instead I have manaully make a dataframe for Japans data and merge it to the result dataframe
url = "https://en.wikipedia.org/wiki/Demographics_of_Japan"
japan = pd.read_html(url, match="Average")

df_japan = pd.DataFrame(japan[0])
df_japan.columns = df_japan.columns.droplevel()
df_japan = df_japan.iloc[:, :2]
df_japan["Country"] = "Japan"
df_japan.columns.values[1] = 'Population'




In [567]:
df_all = pd.concat([result, df_japan], axis=1)

# Renaming a few columns

df_all.rename(columns={list(df_all)[0]: 'Year'}, inplace=True)
df_all.rename(columns={list(df_all)[1]: 'Population'}, inplace=True)
df_all.rename(columns={list(df_all)[2]: 'Country'}, inplace=True)
df_all.head()


,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country
0,1817,25009000,Demographics_of_Germany,1900.0,41154600,Demography_of_the_United_Kingdom,1900.0,NaN,Demographics_of_France,1900.0,5500000.0,Demographics_of_Canada,1935,127250000.0,Demographics_of_the_United_States,1900.0,32377000.0,Demographics_of_Italy,1900.0,3300000.0,Demographics_of_Switzerland,1900.0,2231000.0,Demographics_of_Norway,1899.0,43400000.0,Japan
1,1818,25369000,Demographics_of_Germany,1901.0,41538200,Demography_of_the_United_Kingdom,1901.0,40710000.0,Demographics_of_France,1901.0,5600000.0,Demographics_of_Canada,1936,128053000.0,Demographics_of_the_United_States,1901.0,32550000.0,Demographics_of_Italy,1901.0,3341000.0,Demographics_of_Switzerland,1901.0,2255000.0,Demographics_of_Norway,1900.0,43847000.0,Japan
2,1819,25733000,Demographics_of_Germany,1902.0,41892700,Demography_of_the_United_Kingdom,1902.0,40810000.0,Demographics_of_France,1902.0,5760000.0,Demographics_of_Canada,1937,128825000.0,Demographics_of_the_United_States,1902.0,32787000.0,Demographics_of_Italy,1902.0,3384000.0,Demographics_of_Switzerland,1902.0,2276000.0,Demographics_of_Norway,1901.0,44359000.0,Japan
3,1820,26101000,Demographics_of_Germany,1903.0,42246600,Demography_of_the_United_Kingdom,1903.0,40910000.0,Demographics_of_France,1903.0,5930000.0,Demographics_of_Canada,1938,129825000.0,Demographics_of_the_United_States,1903.0,33004000.0,Demographics_of_Italy,1903.0,3428000.0,Demographics_of_Switzerland,1903.0,2288000.0,Demographics_of_Norway,1902.0,44964000.0,Japan
4,1821,26473000,Demographics_of_Germany,1904.0,42611400,Demography_of_the_United_Kingdom,1904.0,41000000.0,Demographics_of_France,1904.0,6100000.0,Demographics_of_Canada,1939,130880000.0,Demographics_of_the_United_States,1904.0,33237000.0,Demographics_of_Italy,1904.0,3472000.0,Demographics_of_Switzerland,1904.0,2298000.0,Demographics_of_Norway,1903.0,45546000.0,Japan


In [568]:

df_all.head(5)

,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country
0,1817,25009000,Demographics_of_Germany,1900.0,41154600,Demography_of_the_United_Kingdom,1900.0,NaN,Demographics_of_France,1900.0,5500000.0,Demographics_of_Canada,1935,127250000.0,Demographics_of_the_United_States,1900.0,32377000.0,Demographics_of_Italy,1900.0,3300000.0,Demographics_of_Switzerland,1900.0,2231000.0,Demographics_of_Norway,1899.0,43400000.0,Japan
1,1818,25369000,Demographics_of_Germany,1901.0,41538200,Demography_of_the_United_Kingdom,1901.0,40710000.0,Demographics_of_France,1901.0,5600000.0,Demographics_of_Canada,1936,128053000.0,Demographics_of_the_United_States,1901.0,32550000.0,Demographics_of_Italy,1901.0,3341000.0,Demographics_of_Switzerland,1901.0,2255000.0,Demographics_of_Norway,1900.0,43847000.0,Japan
2,1819,25733000,Demographics_of_Germany,1902.0,41892700,Demography_of_the_United_Kingdom,1902.0,40810000.0,Demographics_of_France,1902.0,5760000.0,Demographics_of_Canada,1937,128825000.0,Demographics_of_the_United_States,1902.0,32787000.0,Demographics_of_Italy,1902.0,3384000.0,Demographics_of_Switzerland,1902.0,2276000.0,Demographics_of_Norway,1901.0,44359000.0,Japan
3,1820,26101000,Demographics_of_Germany,1903.0,42246600,Demography_of_the_United_Kingdom,1903.0,40910000.0,Demographics_of_France,1903.0,5930000.0,Demographics_of_Canada,1938,129825000.0,Demographics_of_the_United_States,1903.0,33004000.0,Demographics_of_Italy,1903.0,3428000.0,Demographics_of_Switzerland,1903.0,2288000.0,Demographics_of_Norway,1902.0,44964000.0,Japan
4,1821,26473000,Demographics_of_Germany,1904.0,42611400,Demography_of_the_United_Kingdom,1904.0,41000000.0,Demographics_of_France,1904.0,6100000.0,Demographics_of_Canada,1939,130880000.0,Demographics_of_the_United_States,1904.0,33237000.0,Demographics_of_Italy,1904.0,3472000.0,Demographics_of_Switzerland,1904.0,2298000.0,Demographics_of_Norway,1903.0,45546000.0,Japan


In [569]:
column = df_all.iloc[:, 2]
# Replace all values equal to a certain value with a new value
column = column.replace(to_replace='Demographics_of_Germany', value='Germany')
# Update the DataFrame with the modified column
df_all.iloc[:, 2] = column


# Changing the country name values
column = df_all.iloc[:, 5]
column = column.replace(to_replace='Demography_of_the_United_Kingdom', value='United Kingdom')
df_all.iloc[:, 5] = column

column = df_all.iloc[:, 8]
column = column.replace(to_replace='Demographics_of_France', value='France')
df_all.iloc[:, 8] = column

column = df_all.iloc[:, 11]
column = column.replace(to_replace='Demographics_of_Canada', value='Canada')
df_all.iloc[:, 11] = column

column = df_all.iloc[:, 14]
column = column.replace(to_replace='Demographics_of_the_United_States', value='United States')
df_all.iloc[:, 14] = column

column = df_all.iloc[:, 17]
column = column.replace(to_replace='Demographics_of_Italy', value='Italy')
df_all.iloc[:, 17] = column

column = df_all.iloc[:, 20]
column = column.replace(to_replace='Demographics_of_Switzerland', value='Switzerland')
df_all.iloc[:, 20] = column

column = df_all.iloc[:, 23]
column = column.replace(to_replace='Demographics_of_Norway', value='Norway')
df_all.iloc[:, 23] = column


# Promblematic values, therefore redefining them. (They had footnote reference numbers)
df_all.iloc[12, 4] = 42373600
df_all.iloc[49, 4] = 50331000
df_all.iloc[50, 4] = 50381500

# Promblematic values, therefore redefining them. (They had footnote reference numbers)
df_all.iloc[82, 12] = 2017
df_all.iloc[83, 12] = 2018
df_all.iloc[85, 12] = 2020
df_all.iloc[86, 12] = 2021



df_all.head()



,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country,Year,Population,Country
0,1817,25009000,Germany,1900.0,41154600,United Kingdom,1900.0,NaN,France,1900.0,5500000.0,Canada,1935,127250000.0,United States,1900.0,32377000.0,Italy,1900.0,3300000.0,Switzerland,1900.0,2231000.0,Norway,1899.0,43400000.0,Japan
1,1818,25369000,Germany,1901.0,41538200,United Kingdom,1901.0,40710000.0,France,1901.0,5600000.0,Canada,1936,128053000.0,United States,1901.0,32550000.0,Italy,1901.0,3341000.0,Switzerland,1901.0,2255000.0,Norway,1900.0,43847000.0,Japan
2,1819,25733000,Germany,1902.0,41892700,United Kingdom,1902.0,40810000.0,France,1902.0,5760000.0,Canada,1937,128825000.0,United States,1902.0,32787000.0,Italy,1902.0,3384000.0,Switzerland,1902.0,2276000.0,Norway,1901.0,44359000.0,Japan
3,1820,26101000,Germany,1903.0,42246600,United Kingdom,1903.0,40910000.0,France,1903.0,5930000.0,Canada,1938,129825000.0,United States,1903.0,33004000.0,Italy,1903.0,3428000.0,Switzerland,1903.0,2288000.0,Norway,1902.0,44964000.0,Japan
4,1821,26473000,Germany,1904.0,42611400,United Kingdom,1904.0,41000000.0,France,1904.0,6100000.0,Canada,1939,130880000.0,United States,1904.0,33237000.0,Italy,1904.0,3472000.0,Switzerland,1904.0,2298000.0,Norway,1903.0,45546000.0,Japan


In [570]:
all = all['Population'].astype('Int64')
# Converting population values to intergers



In [571]:
# Making 9 seperate dataframes in order to concatenate them into a 3 wide data frame

df1 = df_all.iloc[:, :3]
df2 = df_all.iloc[:, 3:6]
df3 = df_all.iloc[:, 6:9]
df4 = df_all.iloc[:, 9:12]
df5 = df_all.iloc[:, 12:15]
df6 = df_all.iloc[:, 15:18]
df7 = df_all.iloc[:, 18:21]
df8 = df_all.iloc[:, 21:24]
df9 = df_all.iloc[:, 24:27]


In [583]:
all = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9])
all.dropna(subset=['Year', 'Population', 'Country'], inplace=True)
all['Year'].astype(int)
all['Population'].astype(int)

col = all["Year"]
col1 = all["Population"]

def remove_decimal(x):
    return int(x)


all["Year"] = col.apply(remove_decimal)
all["Population"] = col1.apply(remove_decimal)




In [592]:
new_df = pd.merge(merged_df, all,  how='left', on = ['Country','Year'])
# Adding one missing value for UK population 2021
new_df.iloc[468, 5] = 67349559


# `Calculating the tax per capita`
def calc(row):
    return (row['Adjusted Total tax revenue'] * 1000000000) / row['Population']

new_df['Tax revenue per capita'] = new_df.apply(calc, axis=1)
new_df['Year'] = new_df['Year'].astype(str)

new_df = new_df.drop(new_df.index[:93])
new_df.head(10)




,Country,Year,Total tax revenue,World Average,Adjusted Total tax revenue,Population,Tax revenue per capita
93,Canada,1980,85.842,0.271184,316.545788,24515667.0,12911.979434
94,France,1980,248.154,0.271184,915.077742,53880000.0,16983.625495
95,Germany,1980,399.342,0.271184,1472.589503,78295000.0,18808.218953
96,Italy,1980,83.964,0.271184,309.620588,56434000.0,5486.419329
97,Japan,1980,277.315,0.271184,1022.610089,116600396.0,8770.211112
98,Norway,1980,27.041,0.271184,99.714763,4086000.0,24404.004566
99,Switzerland,1980,27.674,0.271184,102.048975,6319000.0,16149.545075
100,United Kingdom,1980,188.624,0.271184,695.558500,56329000.0,12348.142160
101,United States,1980,730.672,0.271184,2694.382052,227225000.0,11857.771160
102,Canada,1981,101.083,0.301742,334.998415,24819915.0,13497.162045


In [593]:
import json

# Exporting as a json file
exported_values = list(new_df.T.to_dict().values())
open('Tax revenue per capita data.json', 'w').write(json.dumps(exported_values))


86180